*Reranking with a locally hosted reranker model from HuggingFace*

# Setup the notebook

## Install required libs

In [1]:
!pip install -qqU elasticsearch
!pip install -qqU eland[pytorch]
!pip install -qqU datasets

## Import the required python libraries

In [2]:
import os
from elasticsearch import Elasticsearch, helpers, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time
from datasets import load_dataset
import pandas as pd

## Create an Elasticsearch Python client



## Free Trial
If you don't have an Elasticsearch cluster, or what one to test out. Head over to [cloud.elastic.co](https://cloud.elastic.co/registration?onboarding_token=search&cta=cloud-registration&tech=trial&plcmt=article%20content&pg=search-labs) and sign up. You can sign up and have a serverless project up and running in only a few mintues!

We are using an Elastic Cloud cloud_id and deployment (cluster) API key.

[See this guide](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud) for finding the `cloud_id` and creating an `api_key`

In [ ]:
cloud_id = getpass(prompt="Enter your Elasticsearch Cloud ID: ")
api_key = getpass(prompt="Enter your Elasticsearch API key: ")


es = Elasticsearch(cloud_id=cloud_id, api_key=api_key)

try:
    es.info()
    print("Successfully connected to Elasticsearch!")
except exceptions.ConnectionError as e:
    print(f"Error connecting to Elasticsearch: {e}")

# Ready Elasticsearch

## Hugging Face Reranking Model
Run this cell to:
- Use Eland's `eland_import_hub_model` command to upload the reranking model to Elasticsearch.

For this example we've chosen the [`cross-encoder/ms-marco-MiniLM-L-6-v2`](https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2) text similarity model.
<br><br>
**Note**:
While we are importing the model for use as a reranker, Eland and Elasticsearch do not have a dedicated rerank task type, so we still use `text_similarity`

In [ ]:
model_id = "cross-encoder/ms-marco-MiniLM-L-6-v2"

!eland_import_hub_model \
  --cloud-id $cloud_id \
  --es-api-key $api_key \
  --hub-model-id $model_id \
  --task-type text_similarity

## Create Inference Endpoint
Run this cell to:
- Create an inference Endpoint
- Deploy the reranking model we impoted in the previous section
We need to create an endpoint queries can use for reranking

Key points about the `model_config`
- `service` - in this case `elasticsearch` will tell the inference API to use a locally hosted (in Elasticsearch) model
- `num_allocations` sets the number of allocations to 1
    - Allocations are independent units of work for NLP tasks. Scaling this allows for an increase in concurrent throughput
- `num_threads` - sets the number of threads per allocation to 1
    - Threads per allocation affect the number of threads used by each allocation during inference. Scaling this generally increased the speed of inference requests (to a point).
- `model_id` - This is the id of the model as it is named in Elasticsearch



In [6]:
model_config = {
    "service": "elasticsearch",
    "service_settings": {
        "num_allocations": 1,
        "num_threads": 1,
        "model_id": "cross-encoder__ms-marco-minilm-l-6-v2",
    },
    "task_settings": {"return_documents": True},
}

inference_id = "semantic-reranking"

create_endpoint = es.inference.put(
    inference_id=inference_id, task_type="rerank", body=model_config
)

create_endpoint.body

{'inference_id': 'semantic-reranking',
 'task_type': 'rerank',
 'service': 'elasticsearch',
 'service_settings': {'num_allocations': 1,
  'num_threads': 1,
  'model_id': 'cross-encoder__ms-marco-minilm-l-6-v2'},
 'task_settings': {'return_documents': True}}

### Verify it was created

- Run the two cells in this section to verify:
- The Inference Endpoint has been completed
- The model has been deployed

You should see JSON output with information about the semantic endpoint

In [7]:
check_endpoint = es.inference.get(
    inference_id=inference_id,
)

check_endpoint.body

{'endpoints': [{'inference_id': 'semantic-reranking',
   'task_type': 'rerank',
   'service': 'elasticsearch',
   'service_settings': {'num_allocations': 1,
    'num_threads': 1,
    'model_id': 'cross-encoder__ms-marco-minilm-l-6-v2'},
   'task_settings': {'return_documents': True}}]}

## Create the index mapping

We are going to index the `title` and `abstract` from the dataset.  

In [9]:
index_name = "arxiv-papers"

index_mapping = {
    "mappings": {
        "properties": {"title": {"type": "text"}, "abstract": {"type": "text"}}
    }
}


try:
    es.indices.create(index=index_name, body=index_mapping)
    print(f"Index '{index_name}' created successfully.")
except exceptions.RequestError as e:
    if e.error == "resource_already_exists_exception":
        print(f"Index '{index_name}' already exists.")
    else:
        print(f"Error creating index '{index_name}': {e}")

Index 'arxiv-papers' created successfully.


## Ready the dataset
We are going to use the [CShorten/ML-ArXiv-Papers](https://huggingface.co/datasets/CShorten/ML-ArXiv-Papers) dataset.

## Download Dataset
**Note** You may get a warking *The secret `HF_TOKEN` does not exist in your Colab secrets*.

You can safely ignore this.

In [10]:
dataset = load_dataset("CShorten/ML-ArXiv-Papers")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Index into Elasticsearch

We will loop through the dataset and send batches of rows to Elasticsearch
- This may take a couple minutes depending on your cluster sizing.

In [11]:
def bulk_insert_elasticsearch(dataset, index_name, chunk_size=1000):
    actions = []
    for record in dataset:
        action = {
            "_index": index_name,
            "_source": {"title": record["title"], "abstract": record["abstract"]},
        }
        actions.append(action)

        if len(actions) == chunk_size:
            helpers.bulk(es, actions)
            actions = []

    if actions:
        helpers.bulk(es, actions)


bulk_insert_elasticsearch(dataset["train"], index_name)

# Query with Reranking

This containes a `text_similarity_reranker` retriever which:
1. Uses a Standard Retriever to :
    1. Perform a lexical query against `title field
2. Perform a reranking:
    1. Taks as input the top 100 results from the previous search
      - `"rank_window_size": 100`
    2. Taks as input the query
      - `"inference_text": query`
    3.  Uses our previously created reranking API and model


In [12]:
query = "sparse vector embedding"

# Query scored from score
response_scored = es.search(
    index="arxiv-papers-lexical",
    body={
        "size": 10,
        "retriever": {"standard": {"query": {"match": {"title": query}}}},
        "fields": ["title", "abstract"],
        "_source": False,
    },
)

# Query with Semantic Reranker
response_reranked = es.search(
    index="arxiv-papers-lexical",
    body={
        "size": 10,
        "retriever": {
            "text_similarity_reranker": {
                "retriever": {"standard": {"query": {"match": {"title": query}}}},
                "field": "abstract",
                "inference_id": "semantic-reranking",
                "inference_text": query,
                "rank_window_size": 100,
            }
        },
        "fields": ["title", "abstract"],
        "_source": False,
    },
)

## Print the table comparing the scored and reranked results

In [17]:
titles_scored = [
    paper["fields"]["title"][0] for paper in response_scored.body["hits"]["hits"]
]
titles_reranked = [
    paper["fields"]["title"][0] for paper in response_reranked.body["hits"]["hits"]
]

# Creating a DataFrame
df = pd.DataFrame(
    {"Scored Results": titles_scored, "Reranked Results": titles_reranked}
)

df_styled = df.style.set_properties(**{"text-align": "left"}).set_caption(
    f"Comparison of Scored and Semantic Reranked Results - Query: '{query}'"
)

# Display the table
df_styled

,Scored Results,Reranked Results
0,Compact Speaker Embedding: lrx-vector,Scaling Up Sparse Support Vector Machines by Simultaneous Feature and Sample Reduction
1,Quantum Sparse Support Vector Machines,Spaceland Embedding of Sparse Stochastic Graphs
2,Sparse Support Vector Infinite Push,Elliptical Ordinal Embedding
3,"The Sparse Vector Technique, Revisited",Minimum-Distortion Embedding
4,L-Vector: Neural Label Embedding for Domain Adaptation,Free Gap Information from the Differentially Private Sparse Vector and Noisy Max Mechanisms
5,Spaceland Embedding of Sparse Stochastic Graphs,"Interpolated Discretized Embedding of Single Vectors and Vector Pairs for Classification, Metric Learning and Distance Approximation"
6,Sparse Signal Recovery in the Presence of Intra-Vector and Inter-Vector Correlation,Attention Word Embedding
7,Stable Sparse Subspace Embedding for Dimensionality Reduction,Binary Speaker Embedding
8,Auto-weighted Mutli-view Sparse Reconstructive Embedding,NetSMF: Large-Scale Network Embedding as Sparse Matrix Factorization
9,Embedding Words in Non-Vector Space with Unsupervised Graph Learning,Estimating Vector Fields on Manifolds and the Embedding of Directed Graphs


## Print out Title and Abstract
This will print the title and the abstract for the top 10 results after semantic reranking.

In [14]:
for paper in response_scored.body["hits"]["hits"]:
    print(
        f"Title {paper['fields']['title'][0]} \n  Abstract: {paper['fields']['abstract'][0]}"
    )

Title Compact Speaker Embedding: lrx-vector 
  Abstract:   Deep neural networks (DNN) have recently been widely used in speaker
recognition systems, achieving state-of-the-art performance on various
benchmarks. The x-vector architecture is especially popular in this research
community, due to its excellent performance and manageable computational
complexity. In this paper, we present the lrx-vector system, which is the
low-rank factorized version of the x-vector embedding network. The primary
objective of this topology is to further reduce the memory requirement of the
speaker recognition system. We discuss the deployment of knowledge distillation
for training the lrx-vector system and compare against low-rank factorization
with SVD. On the VOiCES 2019 far-field corpus we were able to reduce the
weights by 28% compared to the full-rank x-vector system while keeping the
recognition rate constant (1.83% EER).

Title Quantum Sparse Support Vector Machines 
  Abstract:   We analyze the com